In [9]:
import os
import matplotlib.pyplot as plt
import scipy.misc
import sklearn.feature_extraction
import numpy as np
import scipy.ndimage
from skimage import measure, io
from skimage import transform
import train_UNET 
import skimage
import json
import datetime
import pickle
import tensorflow as tf

save_results = r'C:\Users\buggyr\Mosaic_Experiments\models'
load_training = r'C:\Users\buggyr\Mosaic_Experiments\data\external\Training Images 3'

In [2]:
import keras
from keras.models import *
from keras.models import Sequential
from keras.layers import Conv2D, Activation, UpSampling2D,Reshape, MaxPooling2D, Dropout, Cropping2D, merge, Input, concatenate, Conv2DTranspose
from keras.optimizers import Adadelta, Nadam, RMSprop
from keras.models import load_model

# from tensorflow.python import debug as tf_debug
# sess = K.get_session()
# sess = tf_debug.LocalCLIDebugWrapperSession(sess)
# K.set_session(sess)

In [3]:
f1 = 64
f2 = 32
f3 = 32
ff = 3
k1 = (9,9)
k2 = (1,1)
k3 = (5,5)

inputs = Input(shape = (None, None, 4))

conv1 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(inputs)
#print ("conv1 shape: "+str(conv1.shape))
conv1 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv1)
#print ("conv1 shape: "+str(conv1.shape))
pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
#print ("pool1 shape: "+str(pool1.shape))

conv2 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool1)
#print ("conv2 shape: "+str(conv2.shape))
conv2 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv2)
#print ("conv2 shape: "+str(conv2.shape))
pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
#print ("pool2 shape: "+str(pool2.shape))

conv3 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool2)
#print ("conv3 shape: "+str(conv3.shape))
conv3 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv3)
#print ("conv3 shape: "+str(conv3.shape))
pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)
#print ("pool3 shape: "+str(pool3.shape))

conv4 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool3)
conv4 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv4)
drop4 = Dropout(0.5)(conv4)
pool4 = MaxPooling2D(pool_size=(2, 2))(drop4)

conv5 = Conv2D(1024, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool4)
conv5 = Conv2D(1024, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv5)
drop5 = Dropout(0.5)(conv5)

up6 = Conv2D(512, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(drop5))
merge6 = concatenate([drop4,up6], axis = -1)
conv6 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge6)
conv6 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv6)

up7 = Conv2D(256, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv6))
merge7 = concatenate([conv3,up7], axis = -1)
conv7 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge7)
conv7 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv7)

up8 = Conv2D(128, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv7))
merge8 = concatenate([conv2,up8], axis = -1)
conv8 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge8)
conv8 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv8)

up9 = Conv2D(64, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv8))
merge9 = concatenate([conv1,up9], axis = -1)
conv9 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge9)
conv9 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv9))
conv9 = Conv2D(3, 3, padding = 'same', kernel_initializer = 'he_normal')(conv9)


model = Model(inputs=[inputs], outputs = [conv9])

model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, None, None, 4) 0                                            
____________________________________________________________________________________________________
conv2d_1 (Conv2D)                (None, None, None, 64 2368        input_1[0][0]                    
____________________________________________________________________________________________________
conv2d_2 (Conv2D)                (None, None, None, 64 36928       conv2d_1[0][0]                   
____________________________________________________________________________________________________
max_pooling2d_1 (MaxPooling2D)   (None, None, None, 64 0           conv2d_2[0][0]                   
___________________________________________________________________________________________

In [7]:
keyname = "_NoActivation"
now=datetime.datetime.now()
save_file=os.path.join(save_results,now.strftime("%Y-%m-%d %H-%M")+keyname)
os.mkdir(save_file)
save_pred = os.path.join(save_file,'Epoch_Predictions')
os.mkdir(save_pred)
save_model = os.path.join(save_file,'Epoch_Models')
os.mkdir(save_model)

with open(os.path.join(save_file,'Model_Summary.txt'),'w') as fh:
    # Pass the file handle in as a lambda function to make it callable
    model.summary(print_fn=lambda x: fh.write(x + '\n'))
    
#rmsprop = RMSprop(lr=0.0005, rho=0.9, epsilon=1e-08, decay=0.0)
optimizer_func = Nadam(lr=0.00002, beta_1=0.9, beta_2=0.999, epsilon=1e-08, schedule_decay=0.004)
loss_func='mse'

early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss', patience=2)
tbCallBack = keras.callbacks.TensorBoard(log_dir=os.path.join(save_file,'TNSR_BRD'), histogram_freq=0, write_graph=True, write_images=True)
csv_logger = keras.callbacks.CSVLogger(os.path.join(save_file,'training.log'), separator=',', append=False)
epoch_predict = train_UNET.Save_predictions(save_pred)
model_checkpoint = keras.callbacks.ModelCheckpoint(os.path.join(save_model,'model.{epoch:02d}-{loss:.2f}.hdf5'), monitor='loss')

model.compile(optimizer=optimizer_func,loss=loss_func)

In [ ]:
fls = len(os.listdir(load_training))

                            #train_dir, patch_size, batch_size
train_generator = train_UNET.train_generator_rgb(load_training,32,32)
#train_generator = train_rgb.train_seq(load_training,32,3)
with tf.device('/gpu:1'):
    history = model.fit_generator(generator = train_generator,steps_per_epoch=3*fls,max_queue_size=10,
                             epochs = 200,callbacks = [tbCallBack,csv_logger,epoch_predict, model_checkpoint])
print(history.history)

model.save(os.path.join(save_file,'DeMos_mod.h5'))

In [ ]:
ptch_sizes = [32, 64, 128]
for ptch_size in ptch_sizes:
    print(str(ptch_size))
    
    now=datetime.datetime.now()
    save_file=os.path.join(save_results,now.strftime("%Y-%m-%d %H-%M"))
    os.mkdir(save_file)
    save_pred = os.path.join(save_file,'Epoch_Predictions')
    os.mkdir(save_pred)

    with open(os.path.join(save_file,'Model_Summary.txt'),'w') as fh:
        # Pass the file handle in as a lambda function to make it callable
        model.summary(print_fn=lambda x: fh.write(x + '\n'))

    #rmsprop = RMSprop(lr=0.0005, rho=0.9, epsilon=1e-08, decay=0.0)
    optimizer_func = Nadam(lr=0.00002, beta_1=0.9, beta_2=0.999, epsilon=1e-08, schedule_decay=0.004)
    loss_func='mse'

    early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss', patience=2)
    tbCallBack = keras.callbacks.TensorBoard(log_dir=os.path.join(save_file,'TNSR_BRD'), histogram_freq=0, write_graph=True, write_images=True)
    csv_logger = keras.callbacks.CSVLogger(os.path.join(save_file,'training.log'), separator=',', append=False)
    epoch_predict = train_rgb.Save_predictions(save_pred)


    model.compile(optimizer=optimizer_func,loss=loss_func)

    fls = len(os.listdir(load_training))

                                #train_dir, patch_size, batch_size
    train_generator = train_rgb.train_generator_rgb(load_training,ptch_size,32)
    history = model.fit_generator(generator = train_generator,steps_per_epoch=3*fls,
                                 epochs = 200,callbacks = [tbCallBack,csv_logger,epoch_predict])
    print(history.history)

    model.save(os.path.join(save_file,'DeMos_mod.h5'))

In [ ]:
#Test Kodak
data ={}

kodak_dir = r'C:\Users\buggyr\Mosaic_Experiments\data\interim\Kodak'
ls = len(os.listdir(kodak_dir))

kodak_generator = train_UNET.predict_generator_rgb(kodak_dir)

# k_pred = model.predict_generator(kodak_generator, steps = ls)

res_Kodak = train_UNET.predict_generator(model,kodak_generator,ls,data,5)

data['Kodak_IMGS_PSNR'] = res_Kodak[0]
data['Kodak_IMGS_SSIM'] = res_Kodak[1]
data['Kodak_AVG_PSNR']  = res_Kodak[2]
data['Kodak_AVG_SSIM']  = res_Kodak[3]

In [ ]:
#Test McManus
McM_dir = r'C:\Users\buggyr\Mosaic_Experiments\data\interim\McM'
ls = len(os.listdir(McM_dir))

McM_generator = train_UNET.predict_generator_rgb(McM_dir)

# k_pred = model.predict_generator(kodak_generator, steps = ls)

res_McM = train_UNET.predict_generator(model,McM_generator,ls,data,5)

data['McM_IMGS_PSNR'] = res_McM[0]
data['McM_IMGS_SSIM'] = res_McM[1]
data['McM_AVG_PSNR']  = res_McM[2]
data['McM_AVG_SSIM']  = res_McM[3]

In [ ]:
#Write Results
data['Parameters'] = {
    'Loss Function': loss_func,
    'Optimizer':str(type(optimizer_func))
}
data['Training Set'] = {
    'Training Path': load_training,
}

with open(os.path.join(save_file,'results.txt'), 'w') as outfile:  
    json.dump(data, outfile,indent=4)

In [ ]:
import importlib
importlib.reload(train_UNET)